# Trabajar con procesos

Al ejecutar un script como un trabajo de Azure Machine Learning, debe definir el contexto de ejecución para la ejecución del trabajo. Una configuración de clave es el destino de proceso en el que se ejecutará el script. Podría tratarse de la estación de trabajo local (en este caso, la instancia de proceso) o un destino de proceso remoto, como el clúster de proceso administrado de Azure Machine Learning que se aprovisiona a petición.

En este cuaderno, creará un clúster de proceso y explorará los destinos de proceso para los trabajos.

## Antes de comenzar

Necesitará la versión más reciente del paquete **azureml-ai-ml** para ejecutar el código en este cuaderno. Ejecute la celda siguiente para comprobar que está instalada.

> **Nota**:
> Si el paquete **azure-ai-ml** no está instalado, ejecute `pip install azure-ai-ml` para instalarlo.

In [ ]:
## Conexión con su área de trabajo

Con los paquetes de SDK necesarios instalados, ya está listo para conectarse al área de trabajo.

Para conectarse a un espacio de trabajo, se necesitan parámetros de identificación: un id. de suscripción, un nombre de grupo de recursos y un nombre de espacio de trabajo. El nombre del grupo de recursos y el nombre del área de trabajo ya se rellenan automáticamente. Solo necesita el identificador de suscripción para completar el comando.

Para buscar los parámetros necesarios, haga clic en la suscripción y el nombre del área de trabajo en la parte superior derecha de Studio. Se abrirá un panel a la derecha.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copie el identificador de suscripción y reemplace **YOUR-SUBSCRIPTION-ID** por el valor que copió. </p>

## Creación de un clúster de proceso

En muchos casos, es posible que los recursos de proceso locales no sean suficientes para procesar un experimento complejo o de larga duración que necesite procesar un gran volumen de datos, y es posible que quiera aprovechar la capacidad de crear y usar dinámicamente recursos de proceso en la nube. Azure Machine Learning admite una variedad de destinos de proceso, que puede definir en el área de trabajo y usarlos para ejecutar trabajos; solo pagará por los recursos cuando los use.

Puede crear un clúster de proceso en [Estudio de Azure Machine Learning](https://ml.azure.com), mediante el SDK de Python o la CLI de Azure. La siguiente celda de código comprueba en su espacio de trabajo la existencia de un clúster de proceso de nombre `aml-cluster`, y si no existe, lo crea.

In [ ]:
Después de crear un clúster de proceso, solo puede cambiar la configuración de:

- `min_instances`: Número mínimo de nodos
- `max_instances`: Número máximo de nodos
- `idle_time_before_scale_down`: Tiempo de inactividad antes de la reducción de escala

Actualmente, el clúster de proceso `aml-cluster` solo puede escalar un máximo de un nodo. Se cambiará a dos, para permitir el proceso paralelo.

In [ ]:
Cuando se actualiza el clúster de proceso, puede comprobar su configuración imprimiendo sus atributos.

## Creación de un script para entrenar un modelo

Para entrenar un modelo, primero creará el script **diabetes_training.py** en la carpeta **src**. El script usa el archivo **diabetes.csv** en la misma carpeta que los datos de entrenamiento.

In [ ]:
## Ejecución de un trabajo en un clúster de proceso

Ahora, ya está listo para ejecutar el trabajo en el clúster de proceso que ha creado.

> **Nota**:
> El trabajo tardará algún tiempo en iniciarse, ya que el clúster de proceso tendrá que escalar de cero a un nodo. Una vez que el clúster de proceso esté listo, se ejecutará el script. Una vez finalizado el trabajo, el clúster de proceso se reducirá verticalmente a cero nodos. Puede revisar el estado del clúster de proceso en la página **Proceso**.

After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [ ]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="cpu-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [ ]:
cpu_cluster = ml_client.compute.get("cpu-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [ ]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [ ]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="cpu-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)